In [2]:
import pytest
import numpy as np
from ampute import MultivariateAmputation
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

## Amputation 

In [11]:
def invidentity(mat):
    r,c=mat.shape
    for i in range(0,r):
        for j in range(0,c):
            if i!=j: mat[i,j]=1
    return mat

# AMPUTATION

data=pd.read_csv("diabetes.csv")

actualData=data.copy()

X = data.values
col=len(data.columns)

my_patterns=np.matrix(invidentity(np.zeros((col,col))).astype(np.int32))
my_patterns[-1,:]=1
my_freqs=np.array((0.07,.15,0.03,.15,.1,.2,.1,.15,.05))
my_prop=0.3
mechanism='MCAR'
current_mechanisms = np.repeat(mechanism, col)

ma = MultivariateAmputation(prop=my_prop, patterns=my_patterns, freqs=my_freqs,mechanisms=current_mechanisms)
incomplete_data = ma.fit_transform(X)

imputMatrix=pd.DataFrame(incomplete_data,columns=data.columns)
imputData=data+imputMatrix
imputData.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,NaN,0.0,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1.0


## Imputation

In [12]:
data=pd.read_csv("diabetes.csv")
#imputData=pd.read_csv("imputedDiabetes40.csv")
imputCopy=imputData.copy()

impLevel=40

# IMPUTE

tobeImputed=imputData.isnull().sum()
tobeImputed=tobeImputed[tobeImputed>0]
colsToBeImputed=tobeImputed.index

In [13]:
colsToBeImputed

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')

In [19]:
imputCol=colsToBeImputed[3]
    
print (imputCol)

# Delete all records with null values
newData=imputData.dropna(subset=[imputCol])
newData.dropna(inplace=True)

# Set the target column to be y
X=newData.drop(imputCol,axis=1).values
y=newData[imputCol].values

dt=DecisionTreeRegressor()

# Select all null valued records in target column as test data
test=imputData[np.isnan(imputData[imputCol])]

X_test=test.drop(imputCol,axis=1)

# Fill all the other data with mean
X_test.fillna(X_test.mean(),inplace=True)
X_test.fillna(0,inplace=True)
X_test=X_test.values

y_test=test[imputCol].values

dt.fit(X,y)

pred=dt.predict(X_test)

pred

SkinThickness


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


array([  0.,  23.,  31.,  23.,  34.,  13.,   0.,  23.,  31.,   0.,  18.,
        27.,  40.,  46.,  17.,  18.,  24.,  19.,  43.,  26.,  19.,  52.,
        27.])

In [20]:
# Update Data
replacementIndex=imputData[imputData[imputCol].isnull()].index

for i,ri in enumerate(replacementIndex):
    imputData.loc[ri,imputCol]=pred[i]
    
imputData.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1.0
